# Genetická optimalizácia (GO) problému obchodného cestujúceho (TSP)

Tento notebook sa zaoberá GO (Genetic optimization) na TSP (Travelling salesman problem). Špeciálna pozornosť je venovaná skúmaniu crossover metód a ich porovnaniu. Súčasťou práce je zavedenie 4 nových crossover metód a ich porovnanie s metódami implementovanými v rámci kurzu.

## Ciele práce 

1. Definovanie nových crossover metód.
2. Hľadanie optimálnych kombinácii crossover, mutation a correction metód pri GO na TSPGrid(3, 3).
3. Tuning hyper-parametrov $ N, M, T_{1}, T_{2} $.
4. Zhodnotenie vplyvu náhodného generovania počiatočných riešení vs. žravej heuristiky.
5. Záverečné zhodnotenie dosiahnutých výsledkov.

## 1. Nové crossover metódy

Pokiaľ sú rodičia (teda riešenia problému) vyjadrené ako vektory: $[x_{0},...,x_{n-1} ] $ a $[y_{0},...,y_{n-1} ] $, potom dieťa vyjadrené ako: <br>
$[z_{0},...,z_{n-1} ] $ vznikne nasledovne.

<b>MatchCrossoverMax() </b>

Pokiaľ $x_{i} = y_{i} $, potom $z_{i} = x_{i} $. <br>
Pokiaľ $x_{i} \neq y_{i} $, potom $z_{i} $ je náhodné celé číslo z intervalu $[0, max(x_{i},y_{i})] $.

<b>MatchCrossoverRnd() </b>

Pokiaľ $x_{i} = y_{i} $, potom $z_{i} = x_{i} $. <br>
Pokiaľ $x_{i} \neq y_{i} $, potom $z_{i} $ je náhodné celé číslo z intervalu $[0, n-i-1] $.

<b>MisMatchCrossoverMax() </b>

Pokiaľ $x_{i} \neq y_{i} $, potom $z_{i} = x_{i} $. <br>
Pokiaľ $x_{i} = y_{i} $, potom $z_{i} $ je náhodné celé číslo z intervalu $[0, y_{i}] $.

<b>MisMatchCrossoverRnd() </b>

Pokiaľ $x_{i} \neq y_{i} $, potom $z_{i} = x_{i} $. <br>
Pokiaľ $x_{i} = y_{i} $, potom $z_{i} $ je náhodné celé číslo z intervalu $[0, n-i-1] $.

In [1]:
# Import path to source directory (bit of a hack in Jupyter)
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, os.path.join('..', 'src')))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
%load_ext autoreload
%autoreload 2

# Work on local computer... add path to source directory
sys.path.append('/home/benjamin/Documents/18heur-2020-master_1/18heur-2020-master/src')

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from objfun_tsp import TSPGrid
tsp = TSPGrid(3, 3)

In [4]:
from heur_go import Crossover, UniformMultipoint, RandomCombination # crossovers
from heur_go import MatchCrossoverMax, MatchCrossoverRnd, MisMatchCrossoverMax, MisMatchCrossoverRnd # new crossovers
from heur_go import GeneticOptimization # heuristics 
from heur_aux import Correction, MirrorCorrection, ExtensionCorrection # corrections 
from heur_aux import CauchyMutation # mutation

## 2. Optimálne kombinácie crossover, mutation a correction metód

V prvej fáze testovania vyskúšame všetky možné kombinácie crossover, mutation a correction metód. Ostatné hyper-parametre heuristiky GO sú zafixované: $N=5, M=15, T_{1}=1.0, T_{2}=0.5 $. <br>

K dispozícii máme: <br>

<b>Crossover metódy:</b> mix Crossover(), RandomCombination(), UniformMultipoint(k), kde parameter $ k $ volí používaťeľ (v našom prípade 1,2,3), MatchCrossoverMax(), MatchCrossoverRnd(), MisMatchCrossoverMax() a MisMatchCrossoverRnd(), <br>
<b>Mutation metóda:</b> CauchyMutation(r), kde parameter $ r $ volí používaťeľ (v našom prípade 1,2,3),  <br>
<b>Correction metódy:</b> vanilla Correction(of), MirrorCorrection(of) a ExtensionCorrection(of).

In [5]:
multipoints = [1, 2, 3]
crossovers = [
    {'crossover': MatchCrossoverMax(), 'name': 'MCM'},
    {'crossover': MatchCrossoverRnd(), 'name': 'MCR'},
    {'crossover': MisMatchCrossoverMax(), 'name': 'MMCM'},
    {'crossover': MisMatchCrossoverRnd(), 'name': 'MMCR'},
    {'crossover': Crossover(), 'name': 'mix'},
    {'crossover': RandomCombination(), 'name': 'rnd'}]
for multipoint in multipoints:
    crossover = {'crossover': UniformMultipoint(multipoint), 'name': 'uni{}'.format(multipoint)}
    crossovers.append(crossover)

corrections = [
    {'correction': Correction(tsp), 'name': 'vanilla'},
    {'correction': MirrorCorrection(tsp), 'name': 'mirror'},
    {'correction': ExtensionCorrection(tsp), 'name': 'extension'}]

parameters = [2, 3, 1]
mutations = []
for parameter in parameters:
    for correction in corrections:
        mutation = {'mutation': CauchyMutation(r=parameter, correction = correction['correction']), 'name': 'cauchy{}_'
                   .format(parameter)+correction['name']}
        mutations.append(mutation)

In [6]:
NUM_RUNS = 1000
maxeval = 100

In [7]:
results = pd.DataFrame()
for crossover in crossovers:
    for mutation in mutations:
        heur_name = 'GO_mut:({})_cro:{}'.format(mutation['name'], crossover['name'])
        runs = []
        for i in tqdm(range(NUM_RUNS), 'Testing {}'.format(heur_name)):
            run = GeneticOptimization(tsp, maxeval, N=5, M=15, Tsel1=1.0, Tsel2=0.5, 
                                      mutation=mutation['mutation'],
                                      crossover=crossover['crossover']).search()
            run['run'] = i
            run['heur'] = heur_name
            run['mutation'] = mutation['name']
            run['crossover'] = crossover['name']
            runs.append(run)

        res_df = pd.DataFrame(runs, columns=['heur', 'run', 'mutation', 'crossover','best_x', 'best_y', 'neval'])
        results = pd.concat([results, res_df], axis=0)

In [7]:
import dill
dill.load_session('heur_3.db')

In [8]:
def rel(x):
    return len([n for n in x if n < np.inf])/len(x)
def mne(x):
    return np.mean([n for n in x if n < np.inf])
def feo(x):
    return mne(x)/rel(x)

In [9]:
results_pivot = results.pivot_table(
    index=['heur', 'mutation', 'crossover'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot = results_pivot.reset_index()
pd.set_option('display.max_rows', 500)
results_pivot.sort_values(by='feo')


/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,heur,mutation,crossover,feo,mne,rel
24,GO_mut:(cauchy1_vanilla)_cro:uni1,cauchy1_vanilla,uni1,266.118172,58.013761,0.218
18,GO_mut:(cauchy1_vanilla)_cro:MCM,cauchy1_vanilla,MCM,273.333333,57.400000,0.210
23,GO_mut:(cauchy1_vanilla)_cro:rnd,cauchy1_vanilla,rnd,283.431418,60.370892,0.213
22,GO_mut:(cauchy1_vanilla)_cro:mix,cauchy1_vanilla,mix,290.320951,57.773869,0.199
25,GO_mut:(cauchy1_vanilla)_cro:uni2,cauchy1_vanilla,uni2,296.257894,61.029126,0.206
26,GO_mut:(cauchy1_vanilla)_cro:uni3,cauchy1_vanilla,uni3,302.113583,60.120603,0.199
72,GO_mut:(cauchy3_vanilla)_cro:MCM,cauchy3_vanilla,MCM,308.767347,54.034286,0.175
45,GO_mut:(cauchy2_vanilla)_cro:MCM,cauchy2_vanilla,MCM,323.982784,58.640884,0.181
80,GO_mut:(cauchy3_vanilla)_cro:uni3,cauchy3_vanilla,uni3,334.278435,52.147436,0.156
50,GO_mut:(cauchy2_vanilla)_cro:rnd,cauchy2_vanilla,rnd,339.906196,59.143678,0.174


Pri pohľade na všetky výsledky zoradené podľa používaného <b>feo</b> kritéria zrejme vidíme, že výrazne najlepšou voľbou Correction metódy je vanilla. Z tohto dôvodu sa pri ďalšom testovaní obmedzíme iba na túto metódu. 

In [10]:
results_pivot_vanilla = results_pivot[(results_pivot['mutation'].isin(['cauchy1_vanilla', 'cauchy2_vanilla', 'cauchy3_vanilla']))]
results_pivot_vanilla[(results_pivot_vanilla['crossover'].isin(['MCM', 'MCR']))].sort_values(by='feo')

,heur,mutation,crossover,feo,mne,rel
18,GO_mut:(cauchy1_vanilla)_cro:MCM,cauchy1_vanilla,MCM,273.333333,57.400000,0.210
72,GO_mut:(cauchy3_vanilla)_cro:MCM,cauchy3_vanilla,MCM,308.767347,54.034286,0.175
45,GO_mut:(cauchy2_vanilla)_cro:MCM,cauchy2_vanilla,MCM,323.982784,58.640884,0.181
19,GO_mut:(cauchy1_vanilla)_cro:MCR,cauchy1_vanilla,MCR,378.574951,59.057692,0.156
46,GO_mut:(cauchy2_vanilla)_cro:MCR,cauchy2_vanilla,MCR,385.306972,59.722581,0.155
73,GO_mut:(cauchy3_vanilla)_cro:MCR,cauchy3_vanilla,MCR,433.272633,53.725806,0.124


In [11]:
results_pivot_vanilla = results_pivot[(results_pivot['mutation'].isin(['cauchy1_vanilla', 'cauchy2_vanilla', 'cauchy3_vanilla']))]
results_pivot_vanilla[(results_pivot_vanilla['crossover'].isin(['MMCM', 'MMCR']))].sort_values(by='feo')

,heur,mutation,crossover,feo,mne,rel
47,GO_mut:(cauchy2_vanilla)_cro:MMCM,cauchy2_vanilla,MMCM,423.952887,54.689922,0.129
74,GO_mut:(cauchy3_vanilla)_cro:MMCM,cauchy3_vanilla,MMCM,429.940828,55.892308,0.130
75,GO_mut:(cauchy3_vanilla)_cro:MMCR,cauchy3_vanilla,MMCR,457.724573,47.145631,0.103
20,GO_mut:(cauchy1_vanilla)_cro:MMCM,cauchy1_vanilla,MMCM,477.146814,54.394737,0.114
48,GO_mut:(cauchy2_vanilla)_cro:MMCR,cauchy2_vanilla,MMCR,662.557528,51.679487,0.078
21,GO_mut:(cauchy1_vanilla)_cro:MMCR,cauchy1_vanilla,MMCR,688.986338,53.051948,0.077


Z definície nových crossover metód vieme, že MatchCrossoverMax() (MCM) a MatchCrossoverRnd() (MCR), <br> resp. MisMatchCrossoverMax() (MMCM) a MisMatchCrossoverRnd() (MMCR) sú si dosť podobné. <br> 
Z výsledkov prvej fázy testov vidíme, že lepšie pracujú Max metódy, teda MCM a MMCM. <br>

Okrem toho je možné vidieť, že MCM pracuje najlepšie v nastavení cauchy1_vanilla. <br>
MMCM pracuje najlepšie v spolupráci s cauchy2_vanilla. <br>

In [12]:
results_pivot_vanilla[(results_pivot_vanilla['crossover'].isin(['uni1', 'uni2', 'uni3']))].sort_values(by='feo')

,heur,mutation,crossover,feo,mne,rel
24,GO_mut:(cauchy1_vanilla)_cro:uni1,cauchy1_vanilla,uni1,266.118172,58.013761,0.218
25,GO_mut:(cauchy1_vanilla)_cro:uni2,cauchy1_vanilla,uni2,296.257894,61.029126,0.206
26,GO_mut:(cauchy1_vanilla)_cro:uni3,cauchy1_vanilla,uni3,302.113583,60.120603,0.199
80,GO_mut:(cauchy3_vanilla)_cro:uni3,cauchy3_vanilla,uni3,334.278435,52.147436,0.156
53,GO_mut:(cauchy2_vanilla)_cro:uni3,cauchy2_vanilla,uni3,348.412170,58.881657,0.169
51,GO_mut:(cauchy2_vanilla)_cro:uni1,cauchy2_vanilla,uni1,351.523438,56.243750,0.160
79,GO_mut:(cauchy3_vanilla)_cro:uni2,cauchy3_vanilla,uni2,381.822222,57.273333,0.150
78,GO_mut:(cauchy3_vanilla)_cro:uni1,cauchy3_vanilla,uni1,398.183155,54.949275,0.138
52,GO_mut:(cauchy2_vanilla)_cro:uni2,cauchy2_vanilla,uni2,412.448204,63.104575,0.153


In [13]:
results_pivot_vanilla[(results_pivot_vanilla['crossover'].isin(['rnd', 'mix']))].sort_values(by='feo')

,heur,mutation,crossover,feo,mne,rel
23,GO_mut:(cauchy1_vanilla)_cro:rnd,cauchy1_vanilla,rnd,283.431418,60.370892,0.213
22,GO_mut:(cauchy1_vanilla)_cro:mix,cauchy1_vanilla,mix,290.320951,57.773869,0.199
50,GO_mut:(cauchy2_vanilla)_cro:rnd,cauchy2_vanilla,rnd,339.906196,59.143678,0.174
49,GO_mut:(cauchy2_vanilla)_cro:mix,cauchy2_vanilla,mix,345.450680,58.035714,0.168
77,GO_mut:(cauchy3_vanilla)_cro:rnd,cauchy3_vanilla,rnd,356.513379,56.329114,0.158
76,GO_mut:(cauchy3_vanilla)_cro:mix,cauchy3_vanilla,mix,382.717208,55.876712,0.146


Analogicky si zistíme aj pre pôvodné crossover metódy, že s ktorou mutation a correction metódou pracujú najlepšie: <br>

UniformMultipoint(1): cauchy1_vanilla <br>
UniformMultipoint(2): cauchy1_vanilla <br>
UniformMultipoint(3): cauchy1_vanilla <br>
RandomCombination(): cauchy1_vanilla <br>
Crossover(): cauchy1_vanilla <br>

Vybrali sme teda 5 najlepších kombinácii (v zmysle porovnávania crossover metód) z tých, čo boli implementované v rámci kurzu HEUR a vyššie sme vybrali 2 najlepšie kombinácie s novo navrhnutými crossover metódami. Týchto 7 nastavení budeme ďalej testovať pre rôzne hodnoty hyper-parametrov $ N, M, T_{1}, T_{2} $. 

## 3. Tuning hyper-parametrov $ N, M, T_{1}, T_{2} $

In [14]:
N = [3, 5, 10, 15]
T1 = [0.5, 1, 5]
T2 = [0.1, 0.4]

Parameter $ M $ volíme $  M = 3*N $.

In [15]:
crossovers_mutations = [
    {'crossover': UniformMultipoint(1), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'uni1_cauchy1'},
    {'crossover': UniformMultipoint(2), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'uni2_cauchy1'},
    {'crossover': UniformMultipoint(3), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'uni3_cauchy1'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'MCM_cauchy1'},
    {'crossover': MisMatchCrossoverMax(), 'mutation': CauchyMutation(r=2, correction = Correction(tsp)), 'name':
    'MMCM_cauchy3'},
    {'crossover': RandomCombination(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'rnd_cauchy1'},
    {'crossover': Crossover(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'mix_cauchy1'}
]

In [16]:
results_2 = pd.DataFrame()
for n in N:
    for temp1 in T1:
        for temp2 in T2:
            for cr_mut in crossovers_mutations:
                heur_name = 'GO_{}_N:{}_T1:{}_T2:{}'.format(cr_mut['name'], n, temp1, temp2)
                runs = []
                for i in tqdm(range(NUM_RUNS), 'Testing {}'.format(heur_name)):
                    run = GeneticOptimization(tsp, maxeval, N=n, M=3*n, Tsel1=temp1, Tsel2=temp2, 
                                              mutation=cr_mut['mutation'],
                                              crossover=cr_mut['crossover']).search()
                    run['run'] = i
                    run['heur'] = heur_name
                    run['n'] = n
                    run['temp1'] = temp1
                    run['temp2'] = temp2
                    runs.append(run)

                res_df = pd.DataFrame(runs, columns=['heur', 'run', 'n','temp1', 'temp2', 
                                                     'best_x', 'best_y', 'neval'])
                results_2 = pd.concat([results_2, res_df], axis=0)

In [16]:
results_pivot_2 = results_2.pivot_table(
    index=['heur', 'n', 'temp1', 'temp2'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot_2 = results_pivot_2.reset_index()
results_pivot_2.sort_values(by='feo')

/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,heur,n,temp1,temp2,feo,mne,rel
13,GO_MCM_cauchy1_N:3_T1:0.5_T2:0.4,3,0.5,0.4,240.151229,55.234783,0.230
12,GO_MCM_cauchy1_N:3_T1:0.5_T2:0.1,3,0.5,0.1,241.220195,56.927966,0.236
108,GO_uni1_cauchy1_N:3_T1:0.5_T2:0.1,3,0.5,0.1,250.224140,54.799087,0.219
19,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.4,5,0.5,0.4,250.236281,55.802691,0.223
3,GO_MCM_cauchy1_N:10_T1:1_T2:0.4,10,1.0,0.4,250.853340,57.445415,0.229
18,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.1,5,0.5,0.1,252.261047,58.776824,0.233
66,GO_mix_cauchy1_N:5_T1:0.5_T2:0.1,5,0.5,0.1,258.328044,57.090498,0.221
15,GO_MCM_cauchy1_N:3_T1:1_T2:0.4,3,1.0,0.4,260.480967,56.263889,0.216
157,GO_uni3_cauchy1_N:3_T1:0.5_T2:0.4,3,0.5,0.4,262.310845,54.822967,0.209
114,GO_uni1_cauchy1_N:5_T1:0.5_T2:0.1,5,0.5,0.1,263.890479,55.153110,0.209


Z tabuľky výsledkov si môžeme všimnúť, že tuning hyper-parametrov viedol k podstatnému zlepšeniu výsledkov z pohľadu <b> feo </b> kritéria. V predchádzajúcej sérii testov sme sa dostali (v najlepšom prípade) na hodnotu približne 266, zatiaľ čo v tejto sérii sme sa dostali až na hodnotu približne 240 (a niekedy menej je viac :D).

Naviac môžeme konštatovať, že v súčinnosti s tuningom hyper-parametrov dostávame najlepšie výsledky pre crossover metódu <b> MCM </b>, ktorá bola navrhnutá v rámci tohto projektu.

## 4. Náhoda vs. žravá heuristika

V tejto časti sa budeme venovať využitiu žravej heuristiky. Tento princíp bude použitý pri generovaní 1. generácie riešení a to nasledovným spôsobom.

V predchádzajúcom nastavení vznikala táto generácia na základe náhody, teda obchodný cestujúci si nasledujúce mesto vždy vyberal náhodne. V prípade žravého prístupu však bude pristupovať takto. Na začiatku sa nachádza v meste, ktoré je štartovné a vždy rovnaké. Následne sa s pravdepodobnosťou p vyberie do mesta, ktoré je najbližšie a ešte nebolo navštívené. Resp. s pravdepodobnosťou 1-p sa vyberie do náhodného mesta, ktoré nebolo navštívené. Týmto spôsobom postupuje, až kým nenavštívi všetky mestá. Paramater p zaručuje, že riešenia nebudú degenerované.

Tento prístup žravej heuristiky otestujeme tak, že zoberieme 10 najlepších heuristík (vrátane hyper parametrov) z vykonaných experimentov. V týchto heuristikách modifikujeme generovanie 1. generácie riešení žravým spôsobom, ktorý bol popísaný. Pre určenie "ideálnej" hodnoty parametra p budeme testovať 5 rôznych hodnôt a to konkrétne 0.2, 0.4, 0.5, 0.6 a 0.8.  

In [17]:
crossovers_mutations_N_T1_T2 = [
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 3, 
     'T1': 0.5, 'T2': 0.4, 'name': 'MCM_cauchy1_N:3_T1:0.5_T2:0.4'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 3, 
     'T1': 0.5, 'T2': 0.1, 'name': 'MCM_cauchy1_N:3_T1:0.5_T2:0.1'},
    {'crossover': UniformMultipoint(1), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 3, 
     'T1': 0.5, 'T2': 0.1, 'name': 'uni1_cauchy1_N:3_T1:0.5_T2:0.1'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 5, 
     'T1': 0.5, 'T2': 0.4, 'name': 'MCM_cauchy1_N:5_T1:0.5_T2:0.4'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 10, 
     'T1': 1.0, 'T2': 0.4, 'name': 'MCM_cauchy1_N:10_T1:1.0_T2:0.4'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 5, 
     'T1': 0.5, 'T2': 0.1, 'name': 'MCM_cauchy1_N:5_T1:0.5_T2:0.1'},
    {'crossover': Crossover(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 5, 
     'T1': 0.5, 'T2': 0.1, 'name': 'mix_cauchy1_N:5_T1:0.5_T2:0.1'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 3, 
     'T1': 1.0, 'T2': 0.4, 'name': 'MCM_cauchy1_N:3_T1:1.0_T2:0.4'}, 
    {'crossover': UniformMultipoint(3), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 3, 
     'T1': 0.5, 'T2': 0.4, 'name': 'uni3_cauchy1_N:3_T1:0.5_T2:0.4'},
    {'crossover': UniformMultipoint(1), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'N': 3, 
     'T1': 0.5, 'T2': 0.1, 'name': 'uni1_cauchy1_N:3_T1:0.5_T2:0.1'}
]

In [18]:
P = [0.2, 0.4, 0.5, 0.6, 0.8]

In [19]:
from heur_go import GeneticOptimization # heuristics

In [27]:
results_3 = pd.DataFrame()
for p in P:
    for cr_mut in crossovers_mutations_N_T1_T2:
        n = cr_mut['N']
        temp1 = cr_mut['T1']
        temp2 = cr_mut['T2']
        heur_name = 'GO_{}_N:{}_T1:{}_T2:{}_p:{}'.format(cr_mut['name'], n, temp1, temp2, p)
        runs = []
        for i in tqdm(range(NUM_RUNS), 'Testing {}'.format(heur_name)):
            run = GeneticOptimization(tsp, maxeval, N=n, M=3*n, Tsel1=temp1, Tsel2=temp2, 
                                      mutation=cr_mut['mutation'],
                                      crossover=cr_mut['crossover'],
                                      p=p).search()
            # tu to treba opravit
            run['run'] = i
            run['heur'] = heur_name
            run['n'] = n
            run['temp1'] = temp1
            run['temp2'] = temp2
            run['p'] = p
            runs.append(run)

        res_df = pd.DataFrame(runs, columns=['heur', 'run', 'n','temp1', 'temp2', 'p', 
                                             'best_x', 'best_y', 'neval'])
        results_3 = pd.concat([results_3, res_df], axis=0)

In [28]:
results_pivot_3 = results_3.pivot_table(
    index=['heur', 'p', 'temp1', 'temp2'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot_3 = results_pivot_3.reset_index()
results_pivot_3.sort_values(by='feo')

/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,heur,p,temp1,temp2,feo,mne,rel
4,GO_MCM_cauchy1_N:10_T1:1.0_T2:0.4_N:10_T1:1.0_...,0.8,1.0,0.4,60.237708,28.853862,0.4790
3,GO_MCM_cauchy1_N:10_T1:1.0_T2:0.4_N:10_T1:1.0_...,0.6,1.0,0.4,66.469076,30.376368,0.4570
29,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.4_N:5_T1:0.5_T2...,0.8,0.5,0.4,74.700847,31.971963,0.4280
34,GO_mix_cauchy1_N:5_T1:0.5_T2:0.1_N:5_T1:0.5_T2...,0.8,0.5,0.1,77.076543,34.684444,0.4500
24,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.1_N:5_T1:0.5_T2...,0.8,0.5,0.1,77.929920,32.029197,0.4110
2,GO_MCM_cauchy1_N:10_T1:1.0_T2:0.4_N:10_T1:1.0_...,0.5,1.0,0.4,78.214784,32.693780,0.4180
33,GO_mix_cauchy1_N:5_T1:0.5_T2:0.1_N:5_T1:0.5_T2...,0.6,0.5,0.1,87.029491,37.161593,0.4270
23,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.1_N:5_T1:0.5_T2...,0.6,0.5,0.1,90.834977,35.425641,0.3900
28,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.4_N:5_T1:0.5_T2...,0.6,0.5,0.4,94.962229,35.420912,0.3730
39,GO_uni1_cauchy1_N:3_T1:0.5_T2:0.1_N:3_T1:0.5_T...,0.8,0.5,0.1,99.779287,35.870654,0.3595


Pri pohľade na výsledky môžeme konštatovať obrovský progres, keďže pri feo kritériu sme dosiahli hodnotu približne 60. Pre porovnanie, v predchádzajúcich častiach sme sa dostali najlepšie na hodnotu 240. Vo všeobecnosti teda môžeme tento žravý prístup za podstatne lepší než je náhodné generovanie riešení. V drtivej väčšine prípadov môžeme takisto povedať, že zvyšovanie parametra p viedlo k zlepšovaniu výsledkov. Poslednou dôležitou vecou, ktorú by som rád zmienil je fakt, že najlepšie výsledky sme dosahovali predovšetkým pri použití MCM crossover metódy, ktorá bola navrhnutá v rámci tejto práce.

## 5. Záverečné zhodnotenie výsledkov

Z celkového pohľadu hodnotím postup práce ako pozitívny a to z dôvodu, že sme sa postupným tuningom hyper-parametrov, zavedením nových crossover metód či využitím žravého prístupu dostávali z hľadiska výsledkov experimentov ku kvalitnejším heuristikám, ktoré riešili problém obchodného cestujúceho. Najvýraznejšie zlepšenie sme mohli sledovať práve pri zavedení žravého prístupu, ktorý je popísaný v rámci 4. časti tohto projektu. Určite je takisto potrebné zdôrazniť zavedenie nových crossover metód. Tie vznikli práve ako súčasť tohto projektu a predovšetkým metóda MCM dosahovala spomedzi všetkých metód najlepšie výsledky. Na danom probléme prekonala aj metódy preberané v rámci kurzu. 

In [29]:
import dill
dill.dump_session('heur_4.db')